In [1]:
class Node:
    def __init__(self, name, duration):
        self.name = name
        self.duration = duration
        self.dependencies = []
        
        self.es = 0
        self.ef = duration
        self.ls = 0
        self.lf = duration
        
        self.ff = 0
        self.tf = 0
    
    def __str__(self):
        return '''Name: {}, Duration: {}\n
        Early Start: {}, Early Finish: {}\n
        Free Float: {}, Total Float: {}\n
        Late Start: {}, Late Finish: {}\n'''.format(self.name, self.duration, self.es, self.ef,self.ff,self.tf,self.ls, self.lf )
        
    def add_es(self, es):
        if es > self.es:
            self.es = es
            self.ef = es+self.duration
            self.ls = es
            self.lf = es+self.duration
        
    def calc_lf(self):
        if len(self.dependencies) == 0:
            return
        min_ls = float("inf")
        for dep in self.dependencies:
            if min_ls > dep.ls:
                min_ls = dep.ls
        self.lf = min_ls
        self.ls = min_ls-self.duration
        
    def add_dep(self, dep):
        self.dependencies.append(dep)
    
    def free_float(self):
        max_ff = 0
        for dep in self.dependencies:
            ff = dep.es - self.ef
            if max_ff < ff:
                max_ff = ff
        self.ff = max_ff
        return self.ff
    
    def total_float(self):
        self.tf = self.lf-self.ef
        return self.tf
    
    def set_dur(self, duration):
        self.duration = duration
    
    def reset(self):
        self.es = 0
        self.ef = 0
        self.ls = 0
        self.lf = 0
        self.duration = 0
        self.ff = 0
        self.tf = 0
        

In [45]:
def run_algo(activities, edges):
    nodes = {}
    print("Running")
#     Building the network
    for name, duration in activities:
        nodes[name] = Node(name, duration)
    
#     Forward Pass
    for edge in edges:
        nodes[edge[1]].add_es(nodes[edge[0]].ef)
        nodes[edge[0]].add_dep(nodes[edge[1]])
    
#     Backward Pass
    for name, duration in reversed(activities):
        nodes[name].calc_lf()
        nodes[name].free_float()
        nodes[name].total_float()

#     Print the network
    for name, duration in activities:
        print(str(nodes[name]))
        
#     Print the Critical Path
    finish_len = 0
    print("Critcal Path")
    crit_path = []
    for name in nodes:
        if nodes[name].tf == 0:
            print("Critical Path: {}, Free Float: {}".format(name))
            crit_path.append(nodes[name])
            finish_len = nodes[name].lf
    

In [46]:
activities = [("Start", 0), ("A", 6), ("B", 3), ("C", 4), ("D", 5), ("F", 5), ("E", 4),("Finish", 0)]
edges = [("Start", "A"), ("Start", "B"), ("A", "E"), ("B", "C"), ("B", "D"), ("C", "E"), ("C", "F"), ("D", "F"), ("E", "Finish"), ("F", "Finish")]
run_algo(activities, edges)

Running
Name: Start, Duration: 0

        Early Start: 0, Early Finish: 0

        Free Float: 0, Total Float: 0

        Late Start: 0, Late Finish: 0

Name: A, Duration: 6

        Early Start: 0, Early Finish: 6

        Free Float: 1, Total Float: 3

        Late Start: 3, Late Finish: 9

Name: B, Duration: 3

        Early Start: 0, Early Finish: 3

        Free Float: 0, Total Float: 0

        Late Start: 0, Late Finish: 3

Name: C, Duration: 4

        Early Start: 3, Early Finish: 7

        Free Float: 1, Total Float: 1

        Late Start: 4, Late Finish: 8

Name: D, Duration: 5

        Early Start: 3, Early Finish: 8

        Free Float: 0, Total Float: 0

        Late Start: 3, Late Finish: 8

Name: F, Duration: 5

        Early Start: 8, Early Finish: 13

        Free Float: 0, Total Float: 0

        Late Start: 8, Late Finish: 13

Name: E, Duration: 4

        Early Start: 7, Early Finish: 11

        Free Float: 2, Total Float: 2

        Late Start: 9, Late Finish: 1